In [ ]:
import gymnasium as gym
import torch.nn as nn 
import torch

env = gym.make("Humanoid-v5", render_mode="human")


obs, info = env.reset(seed=0)
obs = torch.tensor(obs,dtype=torch.float32)
input_dim = 348
output_dim = 17


class PolicyNetwork(nn.Module):
    def __init__(self,input_dim,output_dim):
        super(PolicyNetwork, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, output_dim),
            
        )
    def forward(self, x):
        return self.model(x)
    
    
       
policy = PolicyNetwork(input_dim, output_dim)

for _ in range(1000):

    action =policy(obs).detach().numpy()                         
    obs, reward, terminated, truncated, info = env.step(action) 
    obs = torch.tensor(obs,dtype=torch.float32) 
    env.render()                                              

    

env.close()


^C
Note: you may need to restart the kernel to use updated packages.
